**_<mark> you can use more cores to make backfill faster : 2,4,8,16,32,64</mark>_**

In [1]:
#%%configure
#{"vCores": 2}

In [2]:
!pip install -q obstore
!pip install -q duckrun

In [3]:
import duckrun
from   psutil import *

In [4]:
# please don't use a workspace name, Lakehouse and semantic_model with an empty space, or the same name of the lakehouse recently deleted
nbr_days_download     =  int(30 * 2 ** ((cpu_count() - 2) / 2))  # or just input your numbers
lh                    = 'power' 
schema                = 'aemo'
semantic_model        = "directlake_on_onelake" 
ws                    =  notebookutils.runtime.context.get("currentWorkspaceName")

In [5]:
compaction_threshold  =  150
sql_folder            = 'https://github.com/djouallah/fabric_demo/raw/refs/heads/main/transformation/'
directlake_model      = "https://raw.githubusercontent.com/djouallah/fabric_demo/refs/heads/main/semantic_model/directlake.bim"
remaining_files       = max(0,nbr_days_download - 60)
Nbr_threads           = (cpu_count()*2)+1

In [6]:
%%time
dr = duckrun.connect( ws,lh,schema, sql_folder, compaction_threshold  )

Connecting to Lakehouse...
Found 9 Delta tables. Attaching as views...

Attached tables (views) in DuckDB:
┌─────────────┐
│    name     │
│   varchar   │
├─────────────┤
│ calendar    │
│ duid        │
│ mstdatetime │
│ price       │
│ price_today │
│ scada       │
│ scada_today │
│ summary     │
└─────────────┘

CPU times: user 1.84 s, sys: 215 ms, total: 2.05 s
Wall time: 7.21 s


In [7]:
nightly =[
              
              ('scrapingv2', (["https://nemweb.com.au/Reports/Current/Daily_Reports/"],["Reports/Current/Daily_Reports/"],nbr_days_download,ws,lh,Nbr_threads)),
              ('price','append',{'ws': ws,'lh':lh}),
              ('scada','append',{'ws': ws,'lh':lh}),
              ('download_excel',("raw/", ws,lh)),
              ('duid','overwrite',{'ws': ws,'lh':lh}),
              ('calendar','ignore'),
              ('mstdatetime','ignore'),
              ('summary__backfill','overwrite')
         ]

intraday = [
              ('scrapingv2', (["http://nemweb.com.au/Reports/Current/DispatchIS_Reports/","http://nemweb.com.au/Reports/Current/Dispatch_SCADA/" ],
                            ["Reports/Current/DispatchIS_Reports/","Reports/Current/Dispatch_SCADA/"],
                             288, ws,lh,Nbr_threads)),
              ('price_today','append',{'ws': ws,'lh':lh}),
              ('scada_today','append',{'ws': ws,'lh':lh}),
              ('duid','ignore',{'ws': ws,'lh':lh}),
              ('summary__incremental', 'append')            
          ]

history_download = [('scrapingv2',(["https://github.com/djouallah/fabric_demo/tree/main/data/archive/*"],["Reports/Current/Daily_Reports/"],remaining_files,ws,lh,Nbr_threads))]

history_process = [('scada','append',{'ws': ws,'lh':lh}),('price','append',{'ws': ws,'lh':lh}),('summary__backfill_archive','append')]

In [8]:
%%time
#create lakehouse if not exists
dr.run([('create_lakehouse_if_not_exists', (lh))])


Task 1/1: create_lakehouse_if_not_exists
Running Python: create_lakehouse_if_not_exists('power',)
✅ Python 'create_lakehouse_if_not_exists' completed

✅ All tasks completed successfully
CPU times: user 59.7 ms, sys: 7.08 ms, total: 66.8 ms
Wall time: 575 ms


True

In [9]:
%%time
#initial load
dr.run(nightly)


Task 1/8: scrapingv2
Running Python: scrapingv2(['https://nemweb.com.au/Reports/Current/Daily_Reports/'], ['Reports/Current/Daily_Reports/'], 30, 'temp', 'power', 5)
https://nemweb.com.au/Reports/Current/Daily_Reports/ - 0 files extracted (all 60 files already downloaded)
✅ Python 'scrapingv2' completed

Task 2/8: price

❌ Task 2 failed: Parser Error: read_csv cannot take NULL list as parameter
CPU times: user 335 ms, sys: 26.4 ms, total: 362 ms
Wall time: 2.34 s


False

In [10]:
%%time
#create  semantic model
dr.run([('deploy_modelv2', (lh,schema,semantic_model,directlake_model))])


Task 1/1: deploy_modelv2
Running Python: deploy_modelv2('power', 'aemo', 'directlake_on_onelake', 'https://raw.githubusercontent.com/djouallah/fabric_demo/refs/heads/main/semantic_model/directlake.bim')
Power BI Semantic Model Deployment

[Step 1/7] Getting workspace information...
✓ Workspace ID: f24e70e8-54ec-491b-9175-2c1aef1138ce

[Step 2/7] Checking if dataset 'directlake_on_onelake' exists...
⚠️  Dataset 'directlake_on_onelake' already exists in this workspace

✓ Dataset 'directlake_on_onelake' already exists - skipping deployment
   Proceeding directly to refresh...

[Step 8/9] Waiting for permission propagation...
   Allowing time for any recent changes to propagate...
   ⏳ 5 seconds remaining...
✓ Wait complete

[Step 9/9] Refreshing semantic model...
   Loading data from lakehouse via DirectLake...
✓ Successfully refreshed semantic model

🎉 Refresh Completed Successfully!

Dataset Name:     directlake_on_onelake
Workspace ID:     f24e70e8-54ec-491b-9175-2c1aef1138ce

✓ Your 

True

In [11]:
%%time
#load today data
dr.run(intraday)


Task 1/5: scrapingv2
Running Python: scrapingv2(['http://nemweb.com.au/Reports/Current/DispatchIS_Reports/', 'http://nemweb.com.au/Reports/Current/Dispatch_SCADA/'], ['Reports/Current/DispatchIS_Reports/', 'Reports/Current/Dispatch_SCADA/'], 288, 'temp', 'power', 5)
Flushed 1 files to disk and updated log Reports/Current/Dispatch_SCADA/download_log.csv
Flushed 1 files to disk and updated log Reports/Current/DispatchIS_Reports/download_log.csv
http://nemweb.com.au/Reports/Current/Dispatch_SCADA/ - 1 files extracted and uploaded
http://nemweb.com.au/Reports/Current/DispatchIS_Reports/ - 1 files extracted and uploaded
✅ Python 'scrapingv2' completed

Task 2/5: price_today
✅ SQL 'price_today' → 'price_today' (append)

Task 3/5: scada_today

❌ Task 3 failed: Parser Error: read_csv cannot take NULL list as parameter
CPU times: user 888 ms, sys: 175 ms, total: 1.06 s
Wall time: 8.62 s


False

**_<u><mark>Files downloaded from github, it is very very slow, but free :)</mark></u>_**

In [12]:
%%time
if remaining_files > 0:
    ## you can turn it off when Historical data is fully loaded
    dr.run(history_download)

CPU times: user 4 µs, sys: 0 ns, total: 4 µs
Wall time: 7.39 µs


In [13]:
if remaining_files > 0:
    dr.run(history_process)

In [14]:
notebookutils.notebook.exit(dr.sql("select count(*) from summary").fetchone()[0])

ExitValue: 128293198